In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Pandas display format
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Import data

In [2]:
entry_df = pd.read_csv("data/entry.csv")
prices_df = pd.read_csv("data/prices.csv")

# Entry

## State level data

How many cities?

In [3]:
print(len(entry_df.city.unique()))

1417


How many states? Why are there 54 states? What are the extra states and why are they being merged into the data?

In [4]:
print(len(entry_df.state.unique()))

54


In [5]:
entry_df.columns

Index(['HD', 'city', 'state', 'LO', 'city.1', 'state.1', 'address', 'city.2',
       'store', 'time', 'url', 'zipcode', 'STATE', 'STATENS', 'STATE_NAME',
       'STUSAB', 'NAME', 'population', 'under44_1', 'under44_2', 'under44_3',
       'older65_1', 'older_65_2', 'income_per_capita', 'industrial_managers',
       'construction_managers', 'farmers', 'realestate',
       'construction_workers', 'state:1', 'place'],
      dtype='object')

HD maybe number of home depot stores, LO equivalent for HD STATENS, maybe CENSUS identifier, under44, number of people under age 44?older65, number of people over 65

In [6]:
entry_df.STATENS.unique()

array([1714934, 1779804, 1779801, 1779795, 1705317, 1102857, 1779799,
       1085497,  897535, 1779796, 1155107,  662849, 1779775, 1629543,
       1779803, 1779784, 1779778, 1779789, 1779798,  294478, 1455989,
       1785533, 1779785, 1325873, 1027616, 1779806, 1779791, 1779779,
        448508, 1779787,  606926, 1779777, 1779805,   68085, 1779808,
       1779802, 1779780,  767982, 1779790, 1779783, 1779786, 1779793,
       1779807, 1779794, 1779781, 1219835, 1779797,  481813, 1779792,
       1779782, 1802710, 1785534, 1802705, 1702382])

In [7]:
entry_df.HD.unique()

array([ 1,  2,  4,  3,  6,  7,  5, 11, 19,  9, 10,  8, 12])

## Place level data

Place Ids don't seem to be unique? Place id == 100 offers an example, it refers to one location in Aberdeen and one in Absecon

In [8]:
entry_df[entry_df.place == 100.000]

,HD,city,state,LO,city.1,state.1,address,city.2,store,time,...,older65_1,older_65_2,income_per_capita,industrial_managers,construction_managers,farmers,realestate,construction_workers,state:1,place
1,1,Aberdeen,53,NaN,NaN,NaN,2213 Port Industrial Rd,Aberdeen,HD,1597100892.168,...,1506.000,1061.000,21544.000,NaN,NaN,NaN,NaN,NaN,53.000,100.000
3,1,Absecon,34,NaN,NaN,NaN,421 Absecon Blvd,Absecon,HD,1597100547.438,...,1062.000,789.000,35991.000,NaN,NaN,NaN,NaN,NaN,34.000,100.000


In [9]:
len(entry_df), len(entry_df.place.unique())

(1584, 1336)

In [10]:
entry_df.HD.unique()

array([ 1,  2,  4,  3,  6,  7,  5, 11, 19,  9, 10,  8, 12])

## Census data

### All null values

 Some columns seem to be null. Which ones? 

In [11]:
entry_df.under44_1

0        569.000
1       1377.000
2      19407.000
3        708.000
4       1963.000
          ...   
1579     910.000
1580   12169.000
1581    1429.000
1582     127.000
1583     210.000
Name: under44_1, Length: 1584, dtype: float64

In [12]:
for col in entry_df.columns[-7:-2]:
    print(col)
    print(entry_df[col].unique())
    print('--------------')

industrial_managers
[nan]
--------------
construction_managers
[nan]
--------------
farmers
[nan]
--------------
realestate
[nan]
--------------
construction_workers
[nan]
--------------


In [13]:
print("Industrial managers")
entry_df.industrial_managers.unique(), entry_df.construction_managers.unique(), entry_df.farmers.unique()

Industrial managers


(array([nan]), array([nan]), array([nan]))

### Population

Population may not have been mergered correctly. see google search results for popluation of Aberdeen, MD and Aberdeen, WA

In [14]:
entry_df[['address', 'city', 'state', 'population']].head(2)

,address,city,state,population
0,979 Beards Hill Road,Aberdeen,24,2723.000
1,2213 Port Industrial Rd,Aberdeen,53,16359.000


In [15]:
entry_df.head(2)

,HD,city,state,LO,city.1,state.1,address,city.2,store,time,...,older65_1,older_65_2,income_per_capita,industrial_managers,construction_managers,farmers,realestate,construction_workers,state:1,place
0,1,Aberdeen,24,NaN,NaN,NaN,979 Beards Hill Road,Aberdeen,HD,1597100429.521,...,18.000,9.000,28641.000,NaN,NaN,NaN,NaN,NaN,24.000,175.000
1,1,Aberdeen,53,NaN,NaN,NaN,2213 Port Industrial Rd,Aberdeen,HD,1597100892.168,...,1506.000,1061.000,21544.000,NaN,NaN,NaN,NaN,NaN,53.000,100.000


In [16]:
entry_df.store.unique()

array(['HD', 'LOW'], dtype=object)

In [17]:
entry_df[['city', 'state','population']]

,city,state,population
0,Aberdeen,24,2723.000
1,Aberdeen,53,16359.000
2,Abilene,48,123676.000
3,Absecon,34,8289.000
4,Acworth,13,22336.000
...,...,...,...
1579,Yulee,12,12616.000
1580,Yuma,4,95573.000
1581,Zachary,22,17056.000
1582,Zanesville,39,2253.000


In [18]:
entry_df.describe()

,HD,state,LO,state.1,time,STATE,STATENS,population,under44_1,under44_2,...,older65_1,older_65_2,income_per_capita,industrial_managers,construction_managers,farmers,realestate,construction_workers,state:1,place
count,1584.000,1584.000,633.000,633.000,1584.000,1584.000,1584.000,1449.000,1443.000,1443.000,...,1443.000,1443.000,1448.000,0.000,0.000,0.000,0.000,0.000,1449.000,1449.000
mean,1.254,27.140,1.273,28.051,1597122062.546,27.140,1503873.972,69471.618,7389.017,10960.044,...,5274.622,3868.283,-888759.182,NaN,NaN,NaN,NaN,NaN,26.834,42784.767
std,0.995,16.300,0.817,16.531,33131.202,16.300,492406.700,146346.169,15760.372,26923.922,...,10172.375,7216.405,24769104.354,NaN,NaN,NaN,NaN,NaN,16.378,24245.218
min,1.000,1.000,1.000,1.000,1597100286.471,1.000,68085.000,0.000,0.000,0.000,...,0.000,0.000,-666666666.000,NaN,NaN,NaN,NaN,NaN,1.000,100.000
25%,1.000,12.000,1.000,12.000,1597100379.109,12.000,1423460.000,12909.000,1053.500,1680.500,...,1089.000,848.500,24346.250,NaN,NaN,NaN,NaN,NaN,12.000,21796.000
50%,1.000,26.000,1.000,29.000,1597100605.555,26.000,1779779.000,30840.000,2869.000,4164.000,...,2502.000,1925.000,29674.000,NaN,NaN,NaN,NaN,NaN,26.000,43930.000
75%,1.000,41.000,1.000,45.000,1597172338.610,41.000,1779796.000,73392.000,7011.000,10665.500,...,5736.000,4398.000,37083.000,NaN,NaN,NaN,NaN,NaN,41.000,62546.000
max,19.000,78.000,10.000,56.000,1597173827.094,78.000,1802710.000,2718555.000,276678.000,536817.000,...,189225.000,137757.000,216416.000,NaN,NaN,NaN,NaN,NaN,72.000,89140.000
